In [43]:
import pandas as pd
import re
import numpy as np

In [44]:
jobs = ['Data scientist', 'Python', 'JS', 'Java','Data scientist', 'Python', 'JS', 'Java']
countrys = ['France','France','France','France', 'USA','USA','USA','USA']
filenames = ['data/'+str.lower(job).replace(' ','')+str.lower(country)+'.csv' for job, country in zip(jobs,countrys) ]

df = pd.concat([pd.read_csv(filename, sep=';', encoding="utf-8-sig", index_col=0) for filename in filenames])

In [45]:
skills = ['Python', 'R', 'SQL', 'NoSQL', 'GIT', 'Spark', 'Flask', 'Streamlit', 'Docker', 'Kubernetes', 'React', 'Machine-Learning', 'Deep Learning', 'NLP', 'Vue', 'Angular', 'Scala', 'PySpark','PowerBI', 'SQLServer', 'Dataiku','Data', 'Keras', 'Tensor-Flow', 'NLU', 'PyTorch', 'Scikit-Learn', 'SAAS', 'Java', 'Scikit learn', 'Hadoop', 'Hive', 'ML','DL', 'Azure', 'AWS']

# contrats = ['CDI','CDD','Apprentissage','Contrat pro','Stage','Intérim','Freelance','Indépendant','Freelance']
            
studylevel_keywords = {'High School':['High School','Baccalauréat'],
                       'Guaduate': ['Université','Universitaire','Formation supérieure','Guaduate','Bac +2','Bac +3','Bachelor'],
                       'Master': ['Master','Master 2', 'Master +2', 'Bac +5', 'Grande école d\'ingénieur','Bac +4/5','M2','Cursus ingénieur', 'engineer','Diplôme ingénieur'],
                       'Doctor': ['Doctorat', 'Doctor'], 
                       'Formation': ['Formation Data Science']
                       }

explevel_keywords = {'Beginer':['Débutant', 'Beginer','No experience','Pas d`\'expérience', '1 an', '0-1 an','0 - 1 an', '0/1 an','0 / 1 an'],
                       'Intermediate': ['Intermédiaire','Intermediate','2 year', '3 year','2-3 year',  '2 ans', '2/3 an', '1-2 an','1-3  years'],
                       'Confirmed': ['Confirmé','Confirmed','3 an','4 an','5 an','3 year','4 year','5 year','3-5 year','3-5 an','3/5 year', '3/5 an'],
                       'Expert': ['Expert','6 year','7 year', '8 year', '9 year','6 an','7 an', '8 an','9 an','10 an','5-10 year','5-10 an','5 - 10 an','5/10 year', '5/10 an'],
                       'Senior': ['Senior', '10+ years', '10+ an', '10 ans +','plus de 10 an']
                       }

In [46]:
def exist_in_array(x, dicto):

    for key in dicto:
        if any(str.lower(term) in str.lower(x) for term in dicto[key]):
            return key

def skill_required(row):

    desc_low = str.lower(row['Description'])
    
    for skill in skills:

        skill_low = str.lower(skill)

        if (skill_low in desc_low) or (skill_low.replace(' ','') in desc_low) or (skill_low.replace('-','') in desc_low)  or (skill_low.replace('-',' ') in desc_low):
            row[skill] = 1

    return row

def searchBy(x,pattern):
    match = re.search(pattern,str.lower(x))
    if match:
        return match.group(0)

def replace_empty_salary(row):

    if (row['Salaire'] == 'None') and (row['SalaryExtract'] is not None): 
        row['Salaire'] = row['SalaryExtract']
    return row      

In [47]:
df['Study required'] = df['Description'].apply(lambda x: exist_in_array(x, studylevel_keywords))
df['Experience required'] = df['Description'].apply(lambda x: exist_in_array(x, explevel_keywords))

df = pd.concat([df, pd.DataFrame(columns=skills)])
df = df.apply(lambda row: skill_required(row), axis=1)

In [48]:
salaryMatch = re.compile(r'((([\$€]\s?\d{1,3}([\.,\s]?\d{3})*[\s]?k?([\.,\s]\d{2})?([\s]?((à)|(au)|(et)|(\/)|(->?)|(to)|(and))?[\s]?[\$€]?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]?\d{2})?)?)|((\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?\s?[\$€]?([\s]?((à)|(au)|(et)|(and)|(\/)|(->?)|(to))?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?)?)[€\$])[\s]?)(\s*(brut)?\s*((par)|(a)|(an)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]*((annually)|(heure)|(jour)|(hebdomadaire)|(mois)|(mensuel)|(hour)|(day)|(month)|(year)|(an))))')

In [303]:
# (?:((([\$€]\s?\d{1,3}([\.,\s]?\d{3})*[\s]?k?([\.,\s]\d{2})?([\s]?((à)|(au)|(et)|(\/)|(->?)|(to)|(and))?[\s]?[\$€]?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]?\d{2})?)?)|((\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?\s?[\$€]?([\s]?((à)|(au)|(et)|(and)|(\/)|(->?)|(to))?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?)?)[€\$])[\s]?)(\s*(brut)?\s*((par)|(a)|(an)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]*((annually)|(heure)|(jour)|(hebdomadaire)|(mois)|(mensuel)|(hour)|(day)|(month)|(year)|(an)))))

In [49]:
df['SalaryExtract'] = df['Description'].apply(lambda x: searchBy(x ,salaryMatch))

In [50]:
df = df.apply(lambda row: replace_empty_salary(row), axis=1)

In [51]:
df.drop('SalaryExtract',axis=1, inplace=True)

In [52]:
df.Salaire.value_counts()

None                          4785
40 000 € - 50 000 € par an      58
50 000 € par an                 43
55 000 € par an                 43
45 000 € par an                 39
                              ... 
$95,000 - $115,000 a year        1
$80 - $90 an hour                1
800 € - 1 200 € par mois         1
700 € par mois                   1
40 000 € - 48 000 € par an       1
Name: Salaire, Length: 263, dtype: int64

In [54]:
df2 = df[df['Salaire']!= 'None']
df2.drop('Type de contrat', axis=1, inplace=True) 

In [55]:
df2.shape

(1121, 43)

In [56]:
df2.Salaire.value_counts()

40 000 € - 50 000 € par an       58
50 000 € par an                  43
55 000 € par an                  43
45 000 € par an                  39
35 000 € - 50 000 € par an       34
40 937 € - 43 418 € par an       33
38 000,00€ à 45 000,00€ /mois    29
48 000 € - 52 000 € par an       27
40 000 € par an                  26
50 000 € - 80 000 € par an       25
35 000 € - 45 000 € par an       18
50 000 € - 65 000 € par an       16
2 470 € - 3 100 € par mois       16
38 000 € - 45 000 € par an       16
40 000 € - 55 000 € par an       16
45 000 € - 55 000 € par an       14
33 000 € - 38 000 € par an       13
28 000 € - 35 000 € par an       12
65 000 € par an                  12
35 000,00€ à 45 000,00€ /mois    11
34 000,00€ à 50 000,00€ /mois    11
40 000 € - 45 000 € par an       10
34 000 € - 44 000 € par an       10
35 000 € - 40 000 € par an       10
33 000 € - 40 000 € par an       10
48 000 € par an                   9
45 000 € - 60 000 € par an        9
40 000 € - 60 000 € par an  

In [57]:
df2 = df2.drop_duplicates()

In [58]:
df2['Classe de salaire'] = np.nan

In [59]:
pd.set_option('display.max_rows',262)
df2.Salaire.value_counts()


40 000 € - 50 000 € par an       22
55 000 € par an                  16
50 000 € par an                  15
35 000 € - 45 000 € par an       11
45 000 € par an                  11
40 000 € par an                   9
45 000 € - 55 000 € par an        8
40 000 € - 60 000 € par an        8
40 000 € - 45 000 € par an        7
40 000 € - 55 000 € par an        6
45 000 € - 60 000 € par an        5
42 000 € par an                   5
500 € par jour                    5
60 000 € - 70 000 € par an        4
50 000 € - 55 000 € par an        4
$70 an hour                       4
35 000 € - 50 000 € par an        4
50 000 € - 60 000 € par an        4
38 000 € - 45 000 € par an        4
$65 an hour                       4
70 000 € par an                   4
30 000 € - 45 000 € par an        3
48 000 € par an                   3
38 000 € par an                   3
550 € par jour                    3
55 000 € - 65 000 € par an        3
35 000 € - 40 000 € par an        3
$60 an hour                 

In [60]:
# def classification_salary(x):
salaire = df2.iloc[3].Salaire

In [61]:
salaire

'40 000 € - 50 000 € par an'

In [62]:
match = re.match('€',salaire)
if match:
    print(match.group(0))


In [31]:
print(re.search('(?:\d*\s?\d{3})\s€\s?par\s?an',salaire))

<re.Match object; span=(11, 26), match='50 000 € par an'>


In [36]:
match = re.findall(r'((([\$€]\s?\d{1,3}([\.,\s]?\d{3})*[\s]?k?([\.,\s]\d{2})?([\s]?((à)|(au)|(et)|(\/)|(->?)|(to)|(and))?[\s]?[\$€]?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]?\d{2})?)?)|((\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?\s?[\$€]?([\s]?((à)|(au)|(et)|(and)|(\/)|(->?)|(to))?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?)?)[€\$])[\s]?)(\s*(brut)?\s*((par)|(a)|(an)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]*((annually)|(heure)|(jour)|(hebdomadaire)|(mois)|(mensuel)|(hour)|(day)|(month)|(year)|(an))))',salaire)
if match:
    print(match)


[('40 000 € - 50 000 € par an', '40 000 € - 50 000 € ', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '40 000 € - 50 000 €', '40 000 € - 50 000 ', ' 000', '', ' - 50 000 ', '-', '', '', '', '', '', '-', '', ' 000', '', 'par an', '', 'par', 'par', '', '', '', '', '', '', '', '', 'an', '', '', '', '', '', '', '', '', '', '', 'an')]


In [90]:
import re
m = re.match(r"(?:\d)(\d)(\d)", "632")
print(m.groups(0))

('3', '2')


In [69]:
matchGroup=re.compile(r'')

if re.search(r'(annually)|(year)|(((par)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]?an)',salaire):
    # if re.search(r'€':)
    # else:
    
elif re.search(r'(month)|(mensuel)|(mois)',salaire):
    pass
elif re.search(r'(week)|(semaine)|(hebdomadaire)',salaire):
    pass
elif re.search(r'(day)|(jour)',salaire):
    pass
elif re.search(r'(hour)|(heur)',salaire):
    pass
else:
    pass

IndentationError: expected an indented block (<ipython-input-69-dc026fe3c339>, line 7)

In [ ]:
((([\$€]\s?\d{1,3}([\.,\s]?\d{3})*[\s]?k?([\.,\s]\d{2})?([\s]?((à)|(au)|(et)|(\/)|(->?)|(to)|(and))?[\s]?[\$€]?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]?\d{2})?)?)|((\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?\s?[\$€]?([\s]?((à)|(au)|(et)|(and)|(\/)|(->?)|(to))?[\s]?\d{1,3}([\.,\s]?\d{3})*\s?k?([\.,\s]\d{2})?)?)[€\$])[\s]?)(\s*(brut)?\s*((par)|(a)|(an)|(per)|(by)|(\/)|(every)|(un)|(une))[\s]*((annually)|(heure)|(jour)|(hebdomadaire)|(mois)|(mensuel)|(hour)|(day)|(month)|(year)|(an))))